In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Dropout, Flatten, Dense
from keras.models import Model
import util_funcs
import pandas as pd
import json

import tensorflow as tf
from keras.saving import load_model
from keras.models import clone_model

In [3]:
def create_model():
  # Define the input with the shape of the input data
  inputs = Input(shape=(206, 192, 3))

  # Create the layers using the Functional API
  x = Conv2D(32, (3, 3), activation='relu')(inputs)
  x = MaxPooling2D(pool_size=(2, 2))(x)

  x = Conv2D(32, (3, 3), activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)

  x = Conv2D(64, (3, 3), activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)

  x = Flatten()(x)
  x = Dense(64, activation='relu')(x)
  x = Dropout(0.5)(x)

  # Output layer
  outputs = Dense(4, activation='softmax')(x)

  # Create the model
  model = Model(inputs=inputs, outputs=outputs)

  # Compile the model
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

def confusion_matrix(model, generator):
    y_true = generator.classes
    y_pred = model.predict(generator)
    y_pred = y_pred.argmax(axis=1)
    return tf.math.confusion_matrix(y_true, y_pred)

In [5]:
data = pd.read_csv('data/labels.csv')
data = data[data['shape'] == '(103, 96)']

img_size = (206,192)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator, validation_generator = util_funcs.make_generators(
    datagen, data, "path", "alteration", img_size, batch_size, 'sparse')

class_indices = validation_generator.class_indices
label_to_class = {v: k for k, v in class_indices.items()}

print(label_to_class)
with open('data/label_to_class.json', 'w') as f:
    json.dump(label_to_class, f)

Found 43870 validated image filenames belonging to 4 classes.
Found 10967 validated image filenames belonging to 4 classes.
{0: 'CR', 1: 'Nope', 2: 'Obl', 3: 'Zcut'}


In [ ]:
model = create_model()

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)]
model.fit(train_generator, epochs=10, validation_data=validation_generator, verbose=1, callbacks=callbacks)
model.save('models/custom_cnn.keras')

In [ ]:
cm = confusion_matrix(model, validation_generator)
 
accuracy = tf.reduce_sum(tf.linalg.diag_part(cm)) / tf.reduce_sum(cm)
precision = tf.linalg.diag_part(cm) / tf.reduce_sum(cm, axis=0)
recall = tf.linalg.diag_part(cm) / tf.reduce_sum(cm, axis=1)
f1 = 2 * precision * recall / (precision + recall)

metrics = {
    "accuracy": accuracy.numpy(),
    "precision": precision.numpy().tolist(),
    "recall": recall.numpy().tolist(),
    "f1": f1.numpy().tolist()
}

with open('data/metrics.json', 'w') as f:
    json.dump(metrics, f)

print(metrics)